# Carga de datos iniciales en bruto

In [6]:
from scrapy.selector import Selector
from sqlalchemy.orm import sessionmaker, mapper
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine, MetaData, Table

engine = create_engine('sqlite:///ockg/rawcursos.sqlite', 
                       echo=False)

Base = declarative_base(engine)

class Raw:
    pass

metadata = MetaData(engine)
mapper_aux = Table('rawcursos', metadata, autoload=True, autoload_with =  engine)

mapper(Raw, mapper_aux)


Session = sessionmaker(bind=engine)
session = Session()

print(f"All set on {engine.url}")

All set on sqlite:///ockg/rawcursos.sqlite


# Generacion de clases

In [7]:
from model import *
import re

regex = re.compile(r'[\n\r\t]')

OC_URL = "http://opencampus.utpl.edu.ec"

class ContentNotFound(Exception):
    pass

final_cursos = []

cursos = session.query(Raw).all()
cont = 0
errores = []
for curso in cursos:
    print(f"({cont}) {curso.curso}")
    temp_curso = Curso()
    name_cut_index =  curso.curso.find("(")
    if name_cut_index != -1:
        temp_curso.nombre = curso.curso[0:name_cut_index]
    else:
        temp_curso.nombre = curso.curso
    temp_curso.url = curso.url
    if curso.curso.find("Edición") != -1:
        temp_curso.edicion =  curso.curso[curso.curso.find("Edición")+8:-1]
    else:
        temp_curso.edicion = 1
        

    about_aux = Selector(text=curso.raw).xpath('//section[@class="about"]/p/text()').get()
    about_aux = regex.sub(" ", about_aux)
    about_aux = about_aux.lstrip()
    # print(f"[ABOUT]{about_aux}")
    if about_aux is not None:
        temp_curso.about = about_aux
    else:
        errores.append({"type":"about","curso":cont})
    
    try:
        competencias_aux = []
        competencias_xpath_variations = ['//section[@class="competencias"]/ul/li/text()','//section[@class="skills-to-aquire"]/ul/li/text()', '//section[@class="competencias"]/ol/li/text()']
        for variation in competencias_xpath_variations:
            competencias_aux = Selector(text=curso.raw).xpath(variation).getall()
            if len(competencias_aux)  > 0:
                break
        if  len(competencias_aux)   == 0: 
            raise ContentNotFound
                
    except ContentNotFound:
        errores.append({"type":"competencias","curso":cont})
    else:                  
        for i in range(0, len(competencias_aux)):
            competencias_aux[i] = regex.sub(" ", competencias_aux[i])
            competencias_aux[i] = competencias_aux[i].lstrip()
            competencias_aux[i] = competencias_aux[i].rstrip()
                 
        # print(f"[COMPETENCIAS]{competencias_aux}")
        temp_curso.competencias = competencias_aux
    
    try:
        uri = None
        uri_xpath_variations = ['//span[@class="important-dates-item-text course-number"]/text()']
        for variation in uri_xpath_variations:
            uri = Selector(text=curso.raw).xpath(variation).get()
            if uri is not None :
                break
        if uri is  None :
            raise ContentNotFound
                
    except ContentNotFound:
        errores.append({"type":"uri","curso":cont})
    else:                            
        uri = uri.lstrip()
        uri = uri.rstrip()
        
        # print(f"[URI]{uri}")
        temp_curso.uri = uri  
        
    try:
        contenidos = []
        contenidos_xpath_variations = ['//section[@class="contenido"]/ul/li/text()','//section[@class="contenido"]/ol/li/text()','//section[@class="contents"]/ul/li/text()', '//section[@class="contents"]/ol/li/text()']
        for variation in contenidos_xpath_variations:
            contenidos = Selector(text=curso.raw).xpath(variation).getall()
            if len(contenidos)  > 0:
                break
        if  len(contenidos)   == 0: 
            raise ContentNotFound
                
    except ContentNotFound:
        errores.append({"type":"contenidos","curso":cont})
    else:                  
        final_contenidos = []
        for i in range(0, len(contenidos)):
            if len(contenidos[i]) > 0:
                contenidos[i] = regex.sub(" ", contenidos[i])
                contenidos[i] = contenidos[i].lstrip()
                contenidos[i] = contenidos[i].rstrip()
                uri = temp_curso.uri+"Content"+str(i+1)
                # print(f">>> {uri} {contenidos[i]}")
                # TODO fix curso 9
                if len(contenidos[i]) >0:
                    final_contenidos.append(Contenido(uri, i+1, contenidos[i]))
        # print(f"[CONTENIDOS]{final_contenidos}")
        temp_curso.contenidos = final_contenidos
    
    
    try:
        prerequisitos_aux = []
        competencias_xpath_variations = ['//section[@class="prerequisites"]/p/text()', '//section[@class="prerequisitos"]/p/text()', '//section[@class="prerequisites"]/ul/li/text()']
        for variation in competencias_xpath_variations:
            prerequisitos_aux = Selector(text=curso.raw).xpath(variation).getall()
            if len(prerequisitos_aux)  > 0:
                break
        if  len(prerequisitos_aux)   == 0: 
            raise ContentNotFound
                
    except ContentNotFound:
        errores.append({"type":"prerequisitos","curso":cont})
    else:                  
        prerequisitos_final = []
        for i in range(0, len(prerequisitos_aux)):
            prerequisitos_aux_list = prerequisitos_aux[i].splitlines()
            # print(prerequisitos_aux_list)
            for item in prerequisitos_aux_list:
                if len(item) > 2:
                    # print(item)
                    prerequisitos_final.append(item)
                    
        for i in range(0, len(prerequisitos_final)):
            prerequisitos_final[i] = regex.sub(" ", prerequisitos_final[i])
            prerequisitos_final[i] = prerequisitos_final[i].lstrip()
            prerequisitos_final[i] = prerequisitos_final[i].rstrip()
                 
        # print(f"[PREREQUISITOS]{prerequisitos_final}")
        temp_curso.prerequisitos = prerequisitos_final
        
        
        
    try:
        duracion = None
        duracion_xpath_variations = ['//section[@class="duracion"]/p/text()','//section[@class="duración"]/p/text()','//section[@class="duration"]/p/text()']
        for variation in duracion_xpath_variations:
            duracion = Selector(text=curso.raw).xpath(variation).get()
            if duracion is not None :
                break
        if duracion is  None :
            raise ContentNotFound
                
    except ContentNotFound:
        errores.append({"type":"duracion","curso":cont})
    else:                            
        duracion = duracion.lstrip()
        duracion = duracion.rstrip()
        
        # print(f"[DURACION]{duracion}")
        temp_curso.duracion = duracion
        
          
        
    try:
        inicio = None
        inicio_xpath_variations = ['//span[@class="important-dates-item-text start-date"]/text()']
        for variation in inicio_xpath_variations:
            inicio = Selector(text=curso.raw).xpath(variation).get()
            if inicio is not None :
                break
        if inicio is  None :
            raise ContentNotFound
                
    except ContentNotFound:
        errores.append({"type":"inicio","curso":cont})
    else:                            
        inicio = inicio.lstrip()
        inicio = inicio.rstrip()
        
        # print(f"[INICIO]{inicio}")
        temp_curso.inicio = inicio
        
    try:
        fin = None
        fin_xpath_variations = ['//span[@class="important-dates-item-text final-date"]/text()']
        for variation in fin_xpath_variations:
            fin = Selector(text=curso.raw).xpath(variation).get()
            if fin is not None :
                break
        if fin is  None :
            raise ContentNotFound
                
    except ContentNotFound:
        errores.append({"type":"fin","curso":cont})
    else:                            
        fin = fin.lstrip()
        fin = fin.rstrip()
        
        # print(f"[FIN]{fin}")
        temp_curso.fin = fin
        
    try:
        esfuerzo_estimado = None
        esfuerzo_estimado_xpath_variations = ['//span[@class="important-dates-item-text effort"]/text()']
        for variation in esfuerzo_estimado_xpath_variations:
            esfuerzo_estimado = Selector(text=curso.raw).xpath(variation).get()
            if esfuerzo_estimado is not None :
                break
        if esfuerzo_estimado is  None :
            raise ContentNotFound
                
    except ContentNotFound:
        errores.append({"type":"esfuerzo_estimado","curso":cont})
    else:                            
        esfuerzo_estimado = esfuerzo_estimado.lstrip()
        esfuerzo_estimado = esfuerzo_estimado.rstrip()
        
        # print(f"[ESFUERZO_ESTIMADO]{esfuerzo_estimado}")
        temp_curso.esfuerzo_estimado = esfuerzo_estimado
         
    try:
        keywords = None
        keywords_xpath_variations = ['//section[@class="palabras_clave"]/p/text()','//section[@class="keywords"]/p/text()']
        for variation in keywords_xpath_variations:
            keywords = Selector(text=curso.raw).xpath(variation).get()
            if keywords is not None :
                break
        if keywords is  None :
            raise ContentNotFound
                
    except ContentNotFound:
        errores.append({"type":"keywords","curso":cont})
    else:           
        keywords = keywords.lstrip()
        keywords = keywords.rstrip()
        
        if "," in keywords:
            keywords = keywords.split(",")
        elif "-" in  keywords:
            keywords = keywords.split("-") 
        elif ";" in  keywords:
            keywords = keywords.split("-")
        elif "•" in keywords:
            keywords = keywords.replace("•","")
            # keywords = keywords.split(".")
            keywords =   keywords.splitlines()
        else:
            keywords.splitlines()
        
        for i in range(0, len(keywords)):
            keywords[i] = keywords[i].lstrip()
            keywords[i] = keywords[i].rstrip()
            # print(keywords[i])
        # print(f"[KEYWORDS]{keywords}")
        temp_curso.keywords = keywords
    
    
    try:
        docentes = []
        docentes_xpath_variations = ['//article[@class="teacher"]']
        for variation in docentes_xpath_variations:
            docentes = Selector(text=curso.raw).xpath(variation).getall()
            if len(docentes)  > 0:
                break
        if  len(docentes)   == 0: 
            raise ContentNotFound
                
    except ContentNotFound:
        errores.append({"type":"docentes","curso":cont})
    else:                  
        final_docentes = []
        
        # print(len(docentes))
        for docente in docentes:
            docente_aux = Docente()
            try:
                nombre_docente = None
                nombre_docente_xpath_variations = ['//h3/text()']
                for variation in nombre_docente_xpath_variations:
                    nombre_docente = Selector(text=docente).xpath(variation).get()
                    if nombre_docente   is not None:
                        break
                if  nombre_docente is None: 
                    raise ContentNotFound
            except ContentNotFound:
                errores.append({"type":"nombre_docente","curso":cont})
            else:    
                docente_aux.nombre = nombre_docente
                docente_aux.uri = generate_uri(nombre_docente)
                # print(f"[NOMBRE_DOCENTE]{docente_aux.nombre}")
                # print(f"[URI_DOCENTE]{docente_aux.uri}")
                
            try:
                detalle_docente = None
                detalle_docente_xpath_variations = ['//p[@align="justify"]/text()','//p[@style="text-align: justify"]/text()']
                for variation in detalle_docente_xpath_variations:
                    detalle_docente = Selector(text=docente).xpath(variation).get()
                    if detalle_docente is not None:
                        break
                if detalle_docente is None:
                    raise ContentNotFound
            except ContentNotFound:
                errores.append({"type":"detalle_docente","curso":cont})
            else:    
                detalle_docente = regex.sub(" ", detalle_docente)
                detalle_docente = detalle_docente.lstrip()
                detalle_docente = detalle_docente.rstrip()
                docente_aux.detalle = detalle_docente
                # print(f"[DETALLE_DOCENTE]{docente_aux.detalle}")
                
            try:
                email_docente = None
                email_docente_xpath_variations = ['//a/@href']
                for variation in email_docente_xpath_variations:
                    email_docente = Selector(text=docente).xpath(variation).get()
                    if email_docente is not None:
                        break
                if email_docente is None:
                    raise ContentNotFound
            except ContentNotFound:
                errores.append({"type":"email_docente","curso":cont})
            else:    
                trim_email = email_docente.find(":")
                # print(trim_email)
                if  trim_email != -1:
                    docente_aux.email = email_docente[trim_email+1:]
                else:
                    docente_aux.email = email_docente
                # print(f"[EMAIL_DOCENTE]{docente_aux.email}")
      
            try:
                img_docente = None
                img_docente_xpath_variations = ['//img/@src']
                for variation in img_docente_xpath_variations:
                    img_docente = Selector(text=docente).xpath(variation).get()
                    if img_docente is not None:
                        break
                if img_docente is None:
                    raise ContentNotFound
            except ContentNotFound:
                errores.append({"type":"img_docente","curso":cont})
            else:    
                
                docente_aux.img = OC_URL+img_docente
                # print(f"[IMG_DOCENTE]{docente_aux.img}")

          
            
            final_docentes.append(docente_aux)
                
            # docentes[i] = regex.sub(" ", docentes[i])
            # docentes[i] = docentes[i].lstrip()
            # docentes[i] = docentes[i].rstrip()
            # uri = generate_uri(temp_curso.uri+docentes[i])
            # final_docentes.append(Contenido(uri, i, docentes[i]))
            
                 
        # print(f"[DOCENTES]{final_docentes}")
        temp_curso.docentes = final_docentes
    

        
    final_cursos.append(temp_curso)
    cont += 1
    
print("Cursos cargados")
if len(errores) > 5:
    print(errores)
else:
    print("No se han encontrado errores")
    print(f"Se han encontrado {len(final_cursos)} cursos")

(0) Fundamentos matemáticos (curso de apoyo) (Edición 2)
(1) Química general (curso de apoyo) (Edición 2)
(2) Orientación Vocacional (curso de apoyo) (Edición 2)
(3) Biología (Curso de Apoyo) (Edición 2)
(4) Administración I (Edición 2)
(5) Contaminación Atmosférica (Edición 11)
(6) Preparación Específica para la Prueba de Admisión (Curso de Apoyo) (Edición 2)
(7) Razonamiento Abstracto  (Curso de Apoyo) (Edición 2)
(8) Conocimiento ancestral de plantas medicinales (Edición 12)
(9) Métodos alternativos de resolución de conflictos (Edición 6)
(10) Comprensión Lectora (Curso de Apoyo) (Edición 2)
(11) Producción Audiovisual (Edición 6)
(12) Desarrollo comunitario (Edición 12)
(13) Fundamentos informáticos (Edición 2)
(14) Prevención integral del consumo de sustancias (Edición 6)
(15) Manejo y análisis de bases de datos (Edición 3)
(16) Manejo y análisis de bases de datos (Edición 4)
(17) Microbiología Ambiental y Agrícola (Edición 5)
(18) Emprendimiento y generación de ideas (Edición 5)


In [8]:
error = 37
print(cursos[error].curso)
print(cursos[error].raw)

Psicología General (Edición 2)
<section class="container">
    <section class="details">

      <nav aria-label="Acerca de">
        <a href="#" class="active">Visión general</a>
      </nav>

      <div class="inner-wrapper">
            <section class="about">
  <h2>Acerca de este curso</h2>

   <p align="justify">
   Este mooc se estudiará los fenómenos psicológicos esenciales que permiten la explicación del comportamiento y de los procesos mentales, objetos de estudio de la psicología. Los procesos psicológicos básicos son indispensables para la vida, permiten tomar conciencia de sí mismo y del entorno, se encuentran en el origen de cualquier expresión conductual y hacen posible el ajuste del comportamiento a las condiciones y demandas ambientales.


   </p>
</section>

    <section class="skills-to-aquire">
  <h2>Competencias a adquirir</h2>
    <ul>
        <li align="justify">Dominar los fundamentos históricos, biológicos y sociales de la Psicología.</li>
<li align="justify">Com

# Creacion del grafo de cursos 

In [15]:
from rdflib import Graph, URIRef
from rdflib.namespace import RDF, RDFS
from rdflib import Namespace, Literal
# Para los cursos
# Para las propiedades adicionales 
ockb = Namespace("http://127.0.0.1/ockb/course/ontology/")
# Para las clases similares
owl = Namespace("http://www.w3.org/2002/07/owl#")
# Para las fechas de inicio y fin
agrelon = Namespace("https://d-nb.info/standards/elementset/agrelon#")
# Para la duracion del curso
time = Namespace("https://www.w3.org/2006/time#")
# Para los requerimientos
dbpedia_owl = Namespace("http://dbpedia.org/ontology/")
# Para informacion variada
schema = Namespace("http://schema.org/")
# Para los profesores
teach = Namespace("http://linkedscience.org/teach/ns#")
ov = Namespace("http://open.vocab.org/terms/")

RESOURCE_BASE_URI = "http://127.0.0.1/ockb/resources/"

g = Graph()

offer_uri = URIRef(RESOURCE_BASE_URI+"OpenCampusFebrero-Julio")
# Setting up current offer
g.add((offer_uri, RDF.type, ockb.CourseOffer))
g.add((offer_uri, ockb.season, Literal("Febrero-Julio")))
g.add((offer_uri, ockb.offerState, Literal(True)))

# Setting up Course as Entity 
g.add((ockb.Course, RDF.type, ockb.Entity))
g.add((ockb.Course, ockb.synonym, Literal("Curso")))
g.add((ockb.Course, ockb.synonym, Literal("Mooc")))
# Setting up Teacher as Entity
g.add((ockb.Teacher, RDF.type, ockb.Entity))
g.add((ockb.Teacher, ockb.synonym, Literal("Docente")))
g.add((ockb.Teacher, ockb.synonym, Literal("Profesor")))
# Setting up Content as Entity
g.add((ockb.Content, RDF.type, ockb.Entity))
g.add((ockb.Content, ockb.synonym, Literal("Contenido")))
g.add((ockb.Content, ockb.synonym, Literal("Tema")))
g.add((ockb.Content, ockb.synonym, Literal("Tematica")))
# Setting up Content as Entity
g.add((ockb.CourseOffer, RDF.type, ockb.Entity))
g.add((ockb.CourseOffer, ockb.synonym, Literal("Oferta")))
g.add((ockb.CourseOffer, ockb.synonym, Literal("Cursos")))


# Setting up reused ontology
# Clases
g.add((ockb.Course, owl.sameAs, schema.Course))
g.add((ockb.Teacher, owl.sameAs, teach.Teacher))
#g.add((ockb.Content, owl.sameAs, ockb.Entity))
#g.add((ockb.CourseOffer, owl.sameAs, ockb.Entity))
# Properties
# begin and endDate
g.add((ockb.beginDate, RDFS.subPropertyOf, agrelon.hasBeginDate))
g.add((ockb.endDate, RDFS.subPropertyOf, agrelon.hasEndDate))
# Requirements
g.add((ockb.requirement, RDFS.subPropertyOf, schema.coursePrerequisites))
# Name
g.add((ockb.courseName, RDFS.subPropertyOf, schema.name))
g.add((ockb.description, RDFS.subPropertyOf, schema.about))
g.add((ockb.url, RDFS.subPropertyOf, schema.url))
# Edition
g.add((ockb.edition, RDFS.subPropertyOf, schema.version))
g.add((ockb.cost, RDFS.subPropertyOf, dbpedia_owl.cost))
# Duration
# g.add((ockb.duration, RDFS.subPropertyOf, schema.duration))
# Teacher
g.add((ockb.hasTeacher, RDFS.subPropertyOf, agrelon.hasTeacher))
g.add((ockb.teacherName, RDFS.subPropertyOf, schema.additionalName))
g.add((ockb.teacherDetail, RDFS.subPropertyOf, schema.knowsAbout))
g.add((ockb.teacherEmail, RDFS.subPropertyOf, schema.email))
g.add((ockb.teaches, RDFS.subPropertyOf, ov.teaches))
# Picture URL
g.add((ockb.pictureURL, RDFS.subPropertyOf, schema.thumbnailUrl))
# COntent
# g.add((ockb.contentOrder, RDFS.subPropertyOf, agrelon.endDate))
# g.add((ockb.content, RDFS.subPropertyOf, agrelon.endDate))
# EstimatedEffort
g.add((ockb.estimatedEffort, RDFS.subPropertyOf, schema.timeRequired))
# Keywords
g.add((ockb.keyword, RDFS.subPropertyOf, schema.keywords))
# Capacity
# g.add((ockb.capacity, RDFS.subPropertyOf, agrelon.endDate))

for curso in final_cursos:
    # curso_uri = ockb.curso.uri
    curso_uri = URIRef(RESOURCE_BASE_URI+curso.uri)
    print(f"[CURSO]{curso_uri}")
    g.add((offer_uri, ockb.hasCourse, curso_uri))
    g.add((curso_uri, RDF.type, ockb.Course))
    g.add((curso_uri, RDFS.label, Literal(curso.nombre)))
    g.add((curso_uri, ockb.courseName, Literal(curso.nombre)))
    
    g.add((curso_uri, ockb.description, Literal(str(curso.about))))
    g.add((curso_uri, ockb.url, Literal(curso.url)))
    g.add((curso_uri, ockb.edition, Literal(curso.edicion)))
    g.add((curso_uri, ockb.cost, Literal("El curso no tiene ningun costo")))

    g.add((curso_uri, ockb.duration, Literal(curso.duracion)))

    for requisito in curso.prerequisitos:
            if len(requisito) > 0 :
                g.add((curso_uri, ockb.requirement, Literal(requisito))) 

    for docente in curso.docentes:
            docente_uri_name = URIRef(RESOURCE_BASE_URI+docente.uri)
            print(f">>[DOCENTE]{docente_uri_name}")
            g.add((docente_uri_name, RDF.type, ockb.Teacher))
            g.add((curso_uri, ockb.hasTeacher, docente_uri_name))
            g.add((docente_uri_name, ockb.teacherName,  Literal(docente.nombre)))
            g.add((docente_uri_name,RDFS.label,  Literal(docente.nombre)))
            g.add((docente_uri_name, ockb.pictureURL,  Literal(docente.img)))
            g.add((docente_uri_name, ockb.teaches,curso_uri))
            g.add((docente_uri_name, ockb.teacherEmail, Literal(docente.email)))
            g.add((docente_uri_name, ockb.teacherDetail, Literal(docente.detalle)))
    
    for contenido in curso.contenidos:
        contenido_uri = URIRef(RESOURCE_BASE_URI+contenido.uri)
        g.add((curso_uri, RDF.type,ockb.Content))
        g.add((curso_uri, ockb.hasContenido,contenido_uri))
        g.add((contenido_uri, ockb.contentOrder,Literal(contenido.order)))
        g.add((contenido_uri, ockb.content,Literal(contenido.contenido)))
    
    
    g.add((curso_uri, ockb.beginDate, Literal(curso.inicio)))
    g.add((curso_uri, ockb.endDate, Literal(curso.fin)))
    g.add((curso_uri, ockb.estimatedEffort, Literal(curso.esfuerzo_estimado)))
    
    try:
        for keyword in curso.keywords:
            g.add((curso_uri, ockb.keyword, Literal(keyword)))
    except Exception as e:
        print(e)
    
    for compentencia in curso.competencias:
        g.add((curso_uri, ockb.capacity,Literal(compentencia)))
g.serialize(destination='cursos.rdf')

# TODO add data types

# from rdflib import Literal,XSD
# Literal("01", datatype=XSD.int)

print("se ha finalizado")

[CURSO]http://127.0.0.1/ockb/resources/MATHFUND2
>>[DOCENTE]http://127.0.0.1/ockb/resources/LuisAlbertoCuencaMacas
[CURSO]http://127.0.0.1/ockb/resources/GENCHEM2
>>[DOCENTE]http://127.0.0.1/ockb/resources/JoseMiguelAndradeMorocho
[CURSO]http://127.0.0.1/ockb/resources/VOCATORIENT2
>>[DOCENTE]http://127.0.0.1/ockb/resources/MonicaMarinaSinchireCastillo
[CURSO]http://127.0.0.1/ockb/resources/BIOLGAPY2
>>[DOCENTE]http://127.0.0.1/ockb/resources/AnabelCuevaAgila
[CURSO]http://127.0.0.1/ockb/resources/ADMINI2
>>[DOCENTE]http://127.0.0.1/ockb/resources/MireyadelCisneLandacayTorres
[CURSO]http://127.0.0.1/ockb/resources/AIRPOLLUTION11
>>[DOCENTE]http://127.0.0.1/ockb/resources/JoseMiguelGuaman
[CURSO]http://127.0.0.1/ockb/resources/ADMSSAPY2
>>[DOCENTE]http://127.0.0.1/ockb/resources/LuisaGabrielaGonzalezGonzalez
[CURSO]http://127.0.0.1/ockb/resources/ABSRSAPY2
>>[DOCENTE]http://127.0.0.1/ockb/resources/GladysARROBORODAS
[CURSO]http://127.0.0.1/ockb/resources/AKOMP12
>>[DOCENTE]http://127.0.